In [ ]:
file = open("C:/Users/PRADEEP REDDY/Desktop/IIITH-Summer Internship/Telugu-Question-Answering-master/Telugu-Question-Answering-master/Data/Questions/location.txt", "rb")
all_ques = []
for q in file:
    q = q.decode('utf-8').split('\n')[0]
    all_ques.append(q)
file.close()
#print(all_ques)
#print(len(all_ques))
################## Answers for the above questions ############
import os

for qu_in in range(len(all_ques)):
    if os.path.exists('web.txt'):
        os.remove('web.txt')
    ###################################  Question Classification  #########################################
    input_query = all_ques[qu_in] #input("Enter your Telugu Query for Question_Classification: ")
    classes = []
    questions = []
    label_generalization = []
    ans_type = ''
    f = open("Data_for_Question_classification.txt" , "rb")
    for line in f:
        #print(line.decode('utf-8'))
        line = line.decode('utf-8').strip('\n')
        #print(line.split()[0])
        label = (line.split()[0]).split(":")[0]    # accessing the labels for the questions in the data.
        classes.append(label)
        questions.append(line[6:])
        #print(line[6:])
        if(label=="PERS"):
            label_generalization.append(1)         # Generalizing the classes
        elif(label=="LOCA"):
            label_generalization.append(2)
        elif(label=="ORGA"):
            label_generalization.append(3)
        elif(label=="DATE"):
            label_generalization.append(4)
        elif(label=="TIME"):
            label_generalization.append(5)
        elif(label=="PERC"):
            label_generalization.append(6)
        elif(label=="NUMB"):
            label_generalization.append(7)
    #print(classes)
    #print(len(classes))
    #print(questions)
    #print(len(questions))
    #print(label_generalization)
    #print(len(label_generalization))
    
    ################## Training and Testing phase #################
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.model_selection import train_test_split
    from sklearn.neural_network import MLPClassifier
    from sklearn import metrics
    
    test_query = input_query
    vectorizer = TfidfVectorizer()
    questions.append(test_query)
    X = vectorizer.fit_transform(questions)
    #print(vectorizer.get_feature_names())
    #print(X.shape)
    
    X1 = X[:len(label_generalization)]
    Y1 = label_generalization[:len(label_generalization)]
    X_train, X_test, y_train, y_test = train_test_split(X1, Y1, train_size = 0.8, test_size = 0.2, random_state = 48)
    #print(X_test)
    #print(y_test)
    
    classifier_MLP = MLPClassifier(hidden_layer_sizes = (100,80,40,7),activation = 'tanh',solver = 'adam',random_state = 3,alpha = 0.1,batch_size = 71)
    #print(classifier_MLP)
    
    classifier_MLP.fit(X_train,y_train)
    
    test1_query = X[-1].toarray()
    #print(X[-1])   # returns the doument_id, token_id, Tfidf score
    #print(test1_query)
    
    prediction2 = classifier_MLP.predict(X_test)

    print("Accuracy: ",metrics.accuracy_score(y_test,prediction2)*100)
    
    ########################### Predicting the User Input query Class ##################################
    print('\n')

    ip_query_predict = classifier_MLP.predict(test1_query)
    if(ip_query_predict == 1):
        predicted_output_query = "PERSON"
    elif(ip_query_predict == 2):
        predicted_output_query = "LOCATION"
    elif(ip_query_predict == 3):
        predicted_output_query = "ORGANIZATION"
    elif(ip_query_predict == 4):
        predicted_output_query = "DATE"
    elif(ip_query_predict == 5):
        predicted_output_query = "TIME"
    elif(ip_query_predict == 6):
        predicted_output_query = "PERCENTAGE"
    else:
        predicted_output_query = "NUMBER"
            
    #print("\n%s belongs to the class [%s]" %(test_query, predicted_output_query))
        
    ans_type += predicted_output_query


            
            
    #################################### Translation of Telugu query to English ################################
    from googletrans import Translator
    trans = Translator()
    en = trans.translate(input_query,dest = 'en',src = 'te')
    tel_to_eng = en.text
    
    print(str(qu_in+1)+": "+"Telugu to English: ", tel_to_eng)
    
    
    ####################################  Getting the access to the web  #####################################
    
    import requests, webbrowser
    from bs4 import BeautifulSoup
    import re
    
    query1 = tel_to_eng   #input("Enter your search query: ")
    query = query1.lower()
    search_engine1 = 'google' #input("Enter the search engine to search, \"Google\" or \"Bing\": ")
    search_engine = search_engine1.lower()
    no_of_urls = 6 #int(input("Enter the number of search results You required: "))
    
    url_list=[]
    
    def access_web(search_engine,query,url_list):
        res = requests.get('https://' + search_engine + '.com/search?q=' + query)
        soup = BeautifulSoup(res.text, 'lxml')
        
        if (search_engine == 'google'):                    ####### Using Google search engine #######
            search_results = soup.select('div#main > div > div > div > a')
            for i in range(len(search_results)):
                url_list.append(search_results[i].get('href'))
                #webbrowser.open('https://' + search_engine + '.com' + search_results[i].get('href'))
            #print(url_list)
            
        elif (search_engine == 'bing'):                   ####### Using Bing search engine ########
            if not(soup.findAll('li', attrs={'class':'b_algo'})):
                access_web(search_engine,query,url_list)
            for i in soup.find_all('li', class_ = 'b_algo'):
                j = i.find('h2')
                for url in j.find_all('a', href=True):
                    url_list.append(url['href'])
                    #webbrowser.open('https://' + search_engine +'.com' + url['href'])
            #print (url_list)
        else:
            print("Enter the search engines either 'Google' or 'Bing' only......")
        return url_list
    
    url_list = access_web(search_engine,query,url_list)
    
    ########################## Getting the Content from the web ##########################
    Sentences=[]
    url_list1 = url_list[:no_of_urls]
    for i in range(no_of_urls):
        try:
            if (search_engine == 'google'):
                res1 = requests.get('https://' + search_engine + '.com' + url_list1[i])
                res1.raise_for_status()
            else:
                res1 = requests.get(url_list1[i])
                res1.raise_for_status()
            content = BeautifulSoup(res1.text,'lxml')
            
            
            if (i==0):
                f = open('web.txt', 'wb')
                for para in content.findAll(['p']):
                    paragraph = para.text
                    Sentences.append(paragraph.strip())
                    f.write(bytes(paragraph,'utf-8'))
                f.close()
            else:
                f1 = open('web.txt', 'ab')
                for para in content.findAll(['p']):
                    paragraph = para.text
                    Sentences.append(paragraph.strip())
                    f1.write(bytes(paragraph,'utf-8'))
                f1.close()
        except requests.HTTPError as e:
            print (e)
        except IOError as e:
            print(e)
        except ValueError as e:
            print(e)
        except Exception as e:
            print(e)
            #print(" ")
            #raise
    #print(Sentences)
    
    
    
    ############################    Finding the similarity matrix  ##########################
    
    import nltk
    from nltk.tokenize import word_tokenize,sent_tokenize
    a = Sentences
    universal_sentence = ""
    all_tokens = []
    for i in range(len(a)):
        universal_sentence += (a[i] + ' ')
        
    One_sentence = sent_tokenize(universal_sentence)
    for j in One_sentence:
        all_tokens.append(word_tokenize(j))
        
    all_tokens_combined = word_tokenize(str.lower(universal_sentence))
    all_unique_tokens_combined = list(set(all_tokens_combined))
    
    
    ###################################   Integer Encoding of all sentences  ###########################################
    
    #print(all_unique_tokens_combined,"\n\n")
    integer_encoding = {}
    for i,word in enumerate(all_unique_tokens_combined):
        
        integer_encoding[word] = i
    #print("\n\n------------------------------------Integer Encoding---------------------------------------\n\n")
    #print("Integer Encoded words:\n",integer_encoding)
    
    ################################  Integer encoding for each sentence  ########################################
    
    def integer_encoding1(subset_sentence):
        subset_sentence_tokens = word_tokenize(subset_sentence)
        subset_sentence_tokens_encoded = {}
        for i in subset_sentence_tokens:
            j = i
            if (j not in integer_encoding.keys()):
                j+='.'
            if (j not in integer_encoding.keys()):
                integer_encoding[j] = len(integer_encoding)
            subset_sentence_tokens_encoded[j] = integer_encoding[j]
            #subset_sentence_tokens_encoded.append(integer_encoding[i])
        return (subset_sentence_tokens_encoded)
    
    #################################################  Cosine Similarity    #####################################################
    
    import math
    def Cosine_Similarity(vec1, vec2):
        common = set(vec1.keys()) & set(vec2.keys())
        numerator = sum([vec1[x] * vec2[x] for x in common])
        
        sum1 = sum([vec1[x]**2 for x in vec1.keys()])
        sum2 = sum([vec2[x]**2 for x in vec2.keys()])
        denominator = (math.sqrt(sum1) * math.sqrt(sum2))
        if not denominator:
            return(0.0)
        
        else:
            return(float(numerator) / denominator)
        
    text1 = query
    ret_sent = {}
    for k in range(len(One_sentence)):
        text2 = str.lower(One_sentence[k])
        vector1 = integer_encoding1(text1)
        vector2 = integer_encoding1(text2)
        cosine_similar = Cosine_Similarity(vector1, vector2)
        ret_sent[One_sentence[k]] = cosine_similar
    
    #print(ret_sent)
    
    
    ####################################### Picking Most relevant sentences ###################################################
    
    print("\n\n\n\n")
    no_of_relevant_sentences = 30 #int(input("Enter the number of relevant sentences you require:"))
    def sort_dict(d):
        return sorted(d.items(), key = lambda x : x[1], reverse = True)
    most_relevant_sentences = sort_dict(ret_sent)
    #print(most_relevant_sentences)
    
    for l in range(min(no_of_relevant_sentences,len(most_relevant_sentences))):
        print(most_relevant_sentences[l][0],'\n')
        
        
    
    ############################  NER Tagging #######################
    print('\n\n\t\t\t\t--------NER Tagging starts------')
    
    import spacy
    nlp = spacy.load('en_core_web_sm')
    
    possible_entities_dictionary = {}
    possible_entities_dictionary_modified = {}
    
    for j in range(min(no_of_relevant_sentences,len(most_relevant_sentences))):
        entities = []
        text = most_relevant_sentences[j][0]
        doc = nlp(text)
        #print(doc)
        
        for entity in doc.ents:
            entities.append([entity.text, entity.label_])
        #print(entities)
        
        for ne, label in entities:
            if (label not in possible_entities_dictionary.keys()):
                possible_entities_dictionary[label] = [ne]
                
            else:
                possible_entities_dictionary[label].append(ne)
        
        for i in possible_entities_dictionary.keys():
            if(i == 'CARDINAL'):
                possible_entities_dictionary_modified['NUMBER'] = possible_entities_dictionary[i]
            elif(i == 'GPE'):
                possible_entities_dictionary_modified['LOCATION'] = possible_entities_dictionary[i]
            elif(i == 'ORG'):
                possible_entities_dictionary_modified['ORGANIZATION'] = possible_entities_dictionary[i]
            elif(i == 'PERCENT'):
                possible_entities_dictionary_modified['PERCENTAGE'] = possible_entities_dictionary[i]
            else:
                if(i in ['PERSON','DATE','TIME']):
                    possible_entities_dictionary_modified[i] = possible_entities_dictionary[i]
                    
                    
    print('\n',possible_entities_dictionary,'\n\n')
    
    print('\n-----------------------------Possible entities dictionary----------------------------\n')
    
    print(possible_entities_dictionary_modified,'\n\n')
    if (ans_type in possible_entities_dictionary_modified.keys()):
        possible_answers = possible_entities_dictionary_modified[ans_type]
    else:
        possible_answers = ['no answer']
    
    print("Possible answers: ", possible_answers,"\n\n")

    ####################### Converting Best answer to Telugu #########################
    
    from googletrans import Translator     # Works only with Internet
    from nltk.tokenize import word_tokenize
    import math
    
    english_query = tel_to_eng
    possible_entities_dictionary1 = possible_entities_dictionary_modified
    
    all_possible_answers1 = possible_answers[:]
    
    token_words = word_tokenize(english_query.lower())
    
    translator = Translator()
    
    #print(token_words)
    
    
    def Cosine_Similarity1(vec1, vec2):
        common = set(vec1.keys()) & set(vec2.keys())
        #print(common)
        numerator = sum([vec1[x] * vec2[x] for x in common])
        
        sum1 = sum([vec1[x]**2 for x in vec1.keys()])
        sum2 = sum([vec2[x]**2 for x in vec2.keys()])
        denominator = (math.sqrt(sum1) * math.sqrt(sum2))
        if not denominator:
            return(0.0)
        else:
            return(float(numerator) / denominator)
        
    text1 = tel_to_eng
    
    for k in range(len(possible_answers)):
        text2 = str.lower(possible_answers[k])
        vector1 = integer_encoding1(text1.lower())
        #print(vector1)
        vector2 = integer_encoding1(text2)
        #print(vector2)
        cosine_similar1 = Cosine_Similarity1(vector1, vector2)
        #print(cosine_similar1)
        if (cosine_similar1 > 0.0):
            all_possible_answers1.remove(possible_answers[k])
            
    print(all_possible_answers1)
    
    all_possible_answers = all_possible_answers1
    
    dict1 = {}
    for i in all_possible_answers:
        if i not in dict1.keys():
            dict1[i] = 1
        else:
            dict1[i] += 1
    #print(dict1)
    if(len(dict1.values())>0):
        max_count = max(dict1.values())
        for k in dict1.keys():
            if (dict1[k] == max_count):
                Best_answer = k
                break
    else:
        Best_answer = 'no answer'
    
    if(Best_answer.isnumeric()):
        Best_answer1 = Best_answer
        print("Ans: ",Best_answer1)
        
    else:
        a = translator.translate(Best_answer, dest='te',src='auto')
        #print(a)
        Best_answer1 = a.text
        print("Ans: ",Best_answer1)
    
    
    que_trans = all_ques[qu_in]

    if(qu_in == 0):
        write_content = ans_type+':'+' '+que_trans[:]+"@@@"+Best_answer1+'\n'
        my_file = open("C:/Users/PRADEEP REDDY/Desktop/IIITH-Summer Internship/Open Domain Questoin Answering/Week-6/MLP/Location/MLP_loc_30_ans_pred.txt","wb")
        my_file.write(bytes(write_content,'utf-8'))
        print(write_content)
    else:
        write_content = ans_type+':'+' '+que_trans[:]+"@@@"+Best_answer1+'\n'
        my_file = open("C:/Users/PRADEEP REDDY/Desktop/IIITH-Summer Internship/Open Domain Questoin Answering/Week-6/MLP/Location/MLP_loc_30_ans_pred.txt","ab")
        my_file.write(bytes(write_content,'utf-8'))
        print(write_content)